In [15]:
import sys
import os

# Set the main path in the root folder of the project.
sys.path.append(os.path.join('..'))

"import sys\nimport os\n\n# Set the main path in the root folder of the project.\nsys.path.append(os.path.join('..'))"

In [70]:
# Settings for autoreloading.
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
from src.utils.seed import set_random_seed

# Set the random seed for deterministic operations.
SEED = 42
set_random_seed(SEED)

In [74]:
import torch

# Set the device for training and querying the model.
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'The selected device is: "{DEVICE}"')

The selected device is: "cuda"


# Loading the Data

In [75]:
import os

BASE_DATA_DIR = os.path.join('..', 'data', 'metr-la')

In [76]:
import pickle
with open(os.path.join(BASE_DATA_DIR, 'processed', 'scaler.pkl'), 'rb') as f:
    scaler = pickle.load(f)

In [77]:
from src.spatial_temporal_gnn.model import SpatialTemporalGNN
from src.data.data_extraction import get_adjacency_matrix

# Get the adjacency matrix
adj_matrix_structure = get_adjacency_matrix(
    os.path.join(BASE_DATA_DIR, 'raw', 'adj_mx_metr_la.pkl'))

# Get the header of the adjacency matrix, the node indices and the
# matrix itself.
header, node_ids_dict, adj_matrix = adj_matrix_structure

# Get the STGNN and load the checkpoints.
spatial_temporal_gnn = SpatialTemporalGNN(9, 1, 12, 12, adj_matrix, DEVICE, 64)

stgnn_checkpoints_path = os.path.join('..', 'models', 'checkpoints',
                                      'st_gnn_metr_la.pth')

stgnn_checkpoints = torch.load(stgnn_checkpoints_path)
spatial_temporal_gnn.load_state_dict(stgnn_checkpoints['model_state_dict'])

# Set the STGNN in evaluation mode.
spatial_temporal_gnn.eval();

In [78]:
from src.data.data_extraction import get_locations_dataframe

# Get the dataframe containing the latitude and longitude of each sensor.
locations_df = get_locations_dataframe(
    os.path.join(BASE_DATA_DIR, 'raw', 'graph_sensor_locations_metr_la.csv'),
    has_header=True)

In [79]:
# Get the node positions dictionary.
node_pos_dict = { i: id for id, i in node_ids_dict.items() }

In [80]:
import pickle

# Get the data scaler.
with open(os.path.join(BASE_DATA_DIR, 'processed', 'scaler.pkl'), 'rb') as f:
    scaler = pickle.load(f)

In [81]:
import os
import numpy as np

# Get the data and the values predicted by the STGNN.
x_train = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_train.npy'))
y_train = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_train.npy'))
x_train_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_train_time.npy'))
y_train_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_train_time.npy'))

x_val = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_val.npy'))
y_val = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_val.npy'))
x_val_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_val_time.npy'))
y_val_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_val_time.npy'))

x_test = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_test.npy'))
y_test = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_test.npy'))
x_test_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_test_time.npy'))
y_test_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_test_time.npy'))

In [82]:
from src.data.data_processing import get_distance_matrix

if not os.path.exists(
    os.path.join(BASE_DATA_DIR, 'processed', 'distance_matrix.npy')):
    # Build the distance matrix between the nodes.
    distance_matrix = get_distance_matrix(
        adj_matrix,
        locations_df,
        node_pos_dict)

    # Save the distance matrix.
    np.save(os.path.join(BASE_DATA_DIR, 'processed', 'distance_matrix.npy'),
            distance_matrix)

else:
    # Load the distance matrix.
    distance_matrix = np.load(
        os.path.join(BASE_DATA_DIR, 'processed', 'distance_matrix.npy'))

In [120]:
from src.explanation.monte_carlo.explanation import apply_grid_search

apply_grid_search(
    x_train[::10],
    y_train[::10],
    distance_matrix,
    spatial_temporal_gnn,
    scaler,
    n_rollouts_list=[30, 50],
    explanation_size_factor_list=[2, 3],
    cut_size_factor_list=[2],
    exploration_weight_list=[5, 10, 20],
    remove_value_list=[0., -10.])

Testing: cut_size_factor: 2 explanation_size_factor: 2 exploration_weight: 5 n_rollouts: 30 remove_value: 0.0
[100/100] - 547s - MAE: { severe_congestion 3.38 -congestion 1.48 -free_flow 0.587 - total: 1.68 } - RMSE: { severe_congestion 3.93 -congestion 1.81 -free_flow 0.756 - total: 2.01 } - MAPE: { severe_congestion 15.1% -congestion 2.93% -free_flow 0.895% - total: 5.79% } - Average time: 5.47s 

Testing: cut_size_factor: 2 explanation_size_factor: 2 exploration_weight: 5 n_rollouts: 30 remove_value: -10.0
[100/100] - 545s - MAE: { severe_congestion 3.35 -congestion 1.53 -free_flow 0.711 - total: 1.73 } - RMSE: { severe_congestion 4.01 -congestion 1.91 -free_flow 0.892 - total: 2.13 } - MAPE: { severe_congestion 15.4% -congestion 3.07% -free_flow 1.08% - total: 6.06% } - Average time: 5.45s 

Testing: cut_size_factor: 2 explanation_size_factor: 2 exploration_weight: 5 n_rollouts: 50 remove_value: 0.0
[100/100] - 890s - MAE: { severe_congestion 2.59 -congestion 1.36 -free_flow 0.557 

In [123]:
CUT_SIZE_FACTOR = 2
EXPLANATION_SIZE_FACTOR = 3
EXPLORATION_WEIGHT = 10
N_ROLLOUTS = 50

In [124]:
import os
import numpy as np

EXPLAINED_DATA_DIR = os.path.join(BASE_DATA_DIR, 'explained')
os.makedirs(EXPLAINED_DATA_DIR, exist_ok=True)


In [125]:
from src.explanation.monte_carlo.explanation import get_all_explanations


print('Computing the explanations for the training set...')
x_train_explained, y_train_explained, train_scores = get_all_explanations(
    x_train,
    y_train,
    distance_matrix,
    spatial_temporal_gnn,
    scaler,
    n_rollouts=N_ROLLOUTS,
    explanation_size_factor=EXPLANATION_SIZE_FACTOR,
    cut_size_factor=CUT_SIZE_FACTOR,
    exploration_weight=EXPLORATION_WEIGHT,
    remove_value=0.,
    divide_by_traffic_cluster_kind=True)

# Save the explained data.
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_train.npy'), x_train_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_train.npy'), y_train_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'train_scores.npy'), train_scores)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_train_time.npy'), x_train_time)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_train_time.npy'), y_train_time)


Computing the explanations for the training set...
[999/999] - 11263s - MAE: { severe_congestion 2.95 -congestion 1.23 -free_flow 0.561 - total: 1.57 } - RMSE: { severe_congestion 3.61 -congestion 1.54 -free_flow 0.719 - total: 1.94 } - MAPE: { severe_congestion 13.1% -congestion 2.5% -free_flow 0.861% - total: 5.44% } - Average time: 11.3s 


In [126]:
from src.explanation.monte_carlo.explanation import get_all_explanations


print('Computing the explanations for the validation set...')
x_val_explained, y_val_explained, val_scores = get_all_explanations(
    x_val,
    y_val,
    distance_matrix,
    spatial_temporal_gnn,
    scaler,
    n_rollouts=N_ROLLOUTS,
    explanation_size_factor=EXPLANATION_SIZE_FACTOR,
    cut_size_factor=CUT_SIZE_FACTOR,
    exploration_weight=EXPLORATION_WEIGHT,
    remove_value=0.,
    divide_by_traffic_cluster_kind=True)

# Save the explained data.
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_val.npy'), x_val_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_val.npy'), y_val_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'val_scores.npy'), val_scores)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_val_time.npy'), x_val_time)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_val_time.npy'), y_val_time)

Computing the explanations for the validation set...
[198/198] - 2155s - MAE: { severe_congestion 2.94 -congestion 1.39 -free_flow 0.548 - total: 1.58 } - RMSE: { severe_congestion 3.61 -congestion 1.73 -free_flow 0.693 - total: 1.95 } - MAPE: { severe_congestion 13% -congestion 2.83% -free_flow 0.845% - total: 5.41% } - Average time: 10.9s 


In [127]:
from src.explanation.monte_carlo.explanation import get_all_explanations


print('Computing the explanations for the test set...')
x_test_explained, y_test_explained, test_scores = get_all_explanations(
    x_test,
    y_test,
    distance_matrix,
    spatial_temporal_gnn,
    scaler,
    n_rollouts=N_ROLLOUTS,
    explanation_size_factor=EXPLANATION_SIZE_FACTOR,
    cut_size_factor=CUT_SIZE_FACTOR,
    exploration_weight=EXPLORATION_WEIGHT,
    remove_value=0.,
    divide_by_traffic_cluster_kind=True)

# Save the explained data.
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_test.npy'), x_test_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_test.npy'), y_test_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'test_scores.npy'), test_scores)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_test_time.npy'), x_test_time)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_test_time.npy'), y_test_time)

Computing the explanations for the test set...
[300/300] - 3428s - MAE: { severe_congestion 3.41 -congestion 1.29 -free_flow 0.556 - total: 1.73 } - RMSE: { severe_congestion 4.18 -congestion 1.62 -free_flow 0.704 - total: 2.14 } - MAPE: { severe_congestion 16.3% -congestion 2.58% -free_flow 0.854% - total: 6.51% } - Average time: 11.4s 
